<a href="https://colab.research.google.com/github/GwenStacey/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 3

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
#Start with imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [17]:
#Now let's grab our data and have a quick look
#Running this block the first time, we note headers aren't included, we'll go ahead and fix that here
#Sine we found the question marks, we can go ahead and make them na here
column_names = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data',
                 header = None, names = column_names, na_values='?' )
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [5]:
#Confirming the correct number of entries
df.shape

(690, 16)

In [0]:
#Basic check for missing values
df.isnull().sum()
#We didn't see any here
#Now that we've updated our missing values, we can see them here clearly

In [0]:
df['A2'].value_counts()
#Here we can clearly see some question marks in our data frame, indicating missing values

In [13]:
df.dtypes
#A2, A11, A14, and A15 are continuous and need to be converted to floats

A1      object
A2      object
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14     object
A15      int64
A16     object
dtype: object

In [0]:
#It didn't want to do this through a for loop for some reason, so I went a little less elegant
df['A11']=df['A11'].astype('float')
df['A14']=df['A14'].astype('float')
df['A15']=df['A15'].astype('float')
  



In [39]:
df.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11    float64
A12     object
A13     object
A14    float64
A15    float64
A16     object
dtype: object

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
#Splitting up by class
df_approved = df[df['A16']=='+']
df_rejected = df[df['A16']=='-']

In [41]:
#A quick look at both dataframes, to make sure they came out right
df_approved.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1.0,f,g,202.0,0.0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280.0,824.0,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,+


In [42]:
#I'm already curious if a NaN so early in rejected is related to the class
df_rejected.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
70,b,32.33,7.5,u,g,e,bb,1.585,t,f,0.0,t,s,420.0,0.0,-
71,b,34.83,4.0,u,g,d,bb,12.500,t,f,0.0,t,g,NaN,0.0,-
72,a,38.58,5.0,u,g,cc,v,13.500,t,f,0.0,t,g,980.0,0.0,-
73,b,44.25,0.5,u,g,m,v,10.750,t,f,0.0,f,s,400.0,0.0,-
74,b,44.83,7.0,y,p,c,v,1.625,f,f,0.0,f,g,160.0,2.0,-


In [0]:
#The sheer number of values in these columns brought me to bin them. I'm not sure
#If they're values that should be in ranges, but it certainly makes things easier to read
df_rejected['A14 Bins']=pd.cut(df['A14'],10)


In [68]:
df['A16'].value_counts()

-    383
+    307
Name: A16, dtype: int64

In [71]:
#I made a crosstab in hopes of checking if these variables might be dependent on one another
table_a1 = pd.crosstab(index = df['A16'], columns = df['A1'])
table_a1

A1,a,b
A16,,
+,98,206
-,112,262


In [72]:
from scipy.stats import chi2_contingency, ttest_ind

chi2, p, dof, expected = chi2_contingency(table_a1)
print(chi2, p, dof, expected)
#Our pvalue is pretty high here, so it appears there may be some independency here

0.31128326491619945 0.5768937883001117 1 [[ 94.15929204 209.84070796]
 [115.84070796 258.15929204]]


In [0]:
#Now I'll make another table for our chi2 to look at
table_a4 = pd.crosstab(index = df['A16'], columns = df['A4'])

In [74]:
#This one is a little more interesting because it allows us to reject the null hypothesis
#Make it likely that these two are dependent on one another
chi2, p, dof, expected = chi2_contingency(table_a4)
print(chi2, p, expected)

26.234074966202144 2.010680204180363e-06 [[  0.88596491 229.90789474  72.20614035]
 [  1.11403509 289.09210526  90.79385965]]


In [64]:
#Now I take a look at some of our continuous features 
s, p = ttest_ind(df_rejected['A2'], df_approved['A2'], nan_policy = 'omit')
print(s)
print(p)
#Our pvalue is low here, indicating a high likelihood that the means are different

-4.2922156166315535
2.0276637071781407e-05


In [75]:
#And one more for good measure
s, p = ttest_ind(df_rejected['A8'], df_approved['A8'], nan_policy = 'omit')
print(s)
print(p)
#Once again, a very low pvalue, indicating it's likely that there is a difference between the means of the classes

-8.935819983773698
3.6710537401601785e-18


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

So, I actually did a lot of the explaining in the comments of each code block. There, unfortunately for us, isn't a lot of detail here, but we can definitely point the people with real information in certain directions. 
Our first chi2 test had quite a high pvalue of 0.57, indicating there is a high statisical change that those variables are independent of one another, the second however, showed the opposite with an absurdly low pvalue. I could say, with some confidence, that those variables could be interesting to look at.
Our ttests both granted us low pvalues, e-05 and e-18. This doesn't tell US too much, but we could tell the folks providing us this data that it's fairly likely that those variables have an impact on which class you wind up in. Not a guarantee mind you, but with a certain degree of confidence.
To be honest, I didn't find this sprint very challenging, especially compared to the last few days of assignments. To the degree I'm actually worried I've done something wrong. I've spent some time going over the instructions though, and am confident I'm submitting a passing sprint challenge.